# Utility

The utility of a player $p$ against an opponent $q$ was define. In this notebook we explore the form of the utility.


We have studied the numerator and the denominator of $u$ separately. 
Both the numerator and the denominator can be written as a general quadratic
form: 

 $$1/2p^TQp + c^T p$$


where $p$ is an $n \times 1$ column vector of variables, $Q$ is a symetric $n \times n$ matrix and a real-valued, $n$-dimensional vector $c$.

Thus the objective function $u$ is the fraction/ratio of the two quadratic forms.

Information on quadratic programming can be found [here](https://neos-guide.org/content/quadratic-programming, http://www.numerical.rl.ac.uk/people/nimg/qp/qp.html).

In [1]:
import opt_mo
import numpy as np
import sympy as sym

sym.init_printing()

In [2]:
p_1, p_2, p_3, p_4 = sym.symbols("p_1, p_2, p_3, p_4")
q_1, q_2, q_3, q_4 = sym.symbols("q_1, q_2, q_3, q_4")

In [3]:
p = (p_1, p_2, p_3, p_4)
q = (q_1, q_2, q_3, q_4)

In [4]:
S_p = sym.Matrix([[3, 0, 5, 1]])

In [5]:
def pd_utility(S_x, p, q):
    """
    Returns the utility as a function
    of p, q vectors. Based on Press and Dyson, 2012.
    """
    B = opt_mo.make_B(S_x, p, q)
    unit= opt_mo.make_B((1, 1, 1, 1), p, q)
    return  B.det() / unit.det()

In [6]:
utility = pd_utility(S_p, p, q)

**Our formulation.**

In [7]:
numerator = sym.fraction(utility.simplify())[0]
denominator = sym.fraction(utility.simplify())[1]

**Numerator**

In [54]:
ps = np.array([[p_1, p_2, p_3, p_4]])
Q_num = np.array([[numerator.coeff(f1 * f2) for f2 in p] for f1 in p])
c_num = np.array([[numerator.subs({p_2:0, p_3:0, p_4:0}).coeff(p_1)], 
         [numerator.subs({p_1:0, p_3:0, p_4:0}).coeff(p_2)],
         [numerator.subs({p_1:0, p_2:0, p_4:0}).coeff(p_3)],
         [numerator.subs({p_1:0, p_2:0, p_3:0}).coeff(p_4)]])

In [69]:
expr = np.dot(ps, Q_num.dot(ps.T) * 1/2)+ (c_num.T).dot(ps)

ValueError: shapes (1,4) and (1,4) not aligned: 4 (dim 1) != 1 (dim 0)

In [71]:
c_num.shape()

TypeError: 'tuple' object is not callable

In [58]:
constant = numerator.expand() - expr
constant

array([[ -p_1*p_2*q_1*q_2 + 5*p_1*p_2*q_1*q_4 + p_1*p_2*q_1 + p_1*p_2*q_2*q_3 - 5*p_1*p_2*q_3*q_4 - p_1*p_2*q_3 + p_1*p_3*q_1*q_3 - p_1*p_3*q_2*q_3 - 5*p_1*p_4*q_1*q_3 + 5*p_1*p_4*q_3*q_4 + p_1*q_1*q_2 - 5*p_1*q_1*q_4 - p_1*q_1 - p_1*(p_2*(-q_1*q_2 + 5*q_1*q_4 + q_1 + q_2*q_3 - 5*q_3*q_4 - q_3)/2 + p_3*(q_1*q_3 - q_2*q_3)/2 + p_4*(-5*q_1*q_3 + 5*q_3*q_4)/2) - p_1*(q_1*q_2 - 5*q_1*q_4 - q_1) + p_2*p_3*q_1*q_2 - p_2*p_3*q_1*q_3 - 3*p_2*p_3*q_2*q_4 - p_2*p_3*q_2 + 3*p_2*p_3*q_3*q_4 + p_2*p_3*q_3 + 5*p_2*p_4*q_1*q_3 - 5*p_2*p_4*q_1*q_4 - 3*p_2*p_4*q_2*q_3 + 3*p_2*p_4*q_2*q_4 - 2*p_2*p_4*q_3 + 2*p_2*p_4*q_4 - p_2*q_2*q_3 + p_2*q_2 + 5*p_2*q_3*q_4 + p_2*q_3 - 5*p_2*q_4 - p_2*(p_1*(-q_1*q_2 + 5*q_1*q_4 + q_1 + q_2*q_3 - 5*q_3*q_4 - q_3)/2 + p_3*(q_1*q_2 - q_1*q_3 - 3*q_2*q_4 - q_2 + 3*q_3*q_4 + q_3)/2 + p_4*(5*q_1*q_3 - 5*q_1*q_4 - 3*q_2*q_3 + 3*q_2*q_4 - 2*q_3 + 2*q_4)/2) - p_2 + 3*p_3*p_4*q_2*q_3 - 3*p_3*p_4*q_3*q_4 - p_3*q_1*q_2 + p_3*q_2*q_3 + 3*p_3*q_2*q_4 + p_3*q_2 - p_3*q_3 - p_3*(p_1*

In [32]:
Q_num = sym.Matrix([[element.factor() for element in row] for row in Q_num])
c_num = sym.Matrix([element.factor() for element in c_num])

In [100]:
for file, name in zip([Q_num, c_num, constant.simplify()],
                      ['q_numerator', 'c_numerator', 'numerator_constant']):
    
    with open("../tex/{}.tex".format(name), "w") as f:
        f.write("{}".format(sym.latex(file)))

In [101]:
(expr + constant).subs({p_1:0, p_2:0, p_3:0, p_4:0, q_1:1, q_2:1, q_3:1, q_4:1})

**Denominator**

In [102]:
Q_den = np.array([[denominator.coeff(f1 * f2) for f2 in p] for f1 in p])
c_den = np.array([denominator.subs({p_2:0, p_3:0, p_4:0}).coeff(p_1), 
         denominator.subs({p_1:0, p_3:0, p_4:0}).coeff(p_2),
         denominator.subs({p_1:0, p_2:0, p_4:0}).coeff(p_3),
         denominator.subs({p_1:0, p_2:0, p_3:0}).coeff(p_4)])

In [103]:
expr = np.dot(ps, Q_den.dot(ps.T) * 1/2) + np.dot(c_den, ps.T)

In [104]:
den_constant = denominator.expand() - expr
den_constant.simplify()

In [105]:
Q_den = sym.Matrix([[element.factor() for element in row] for row in Q_den])
c_den = sym.Matrix([element.factor() for element in c_den])

In [106]:
for file, name in zip([sym.Matrix(Q_den), sym.Matrix(c_den), den_constant.simplify()],
                      ['q_denominator', 'c_denominator', 'denominator_constant']):
    
    with open("../tex/{}.tex".format(name), "w") as f:
        f.write("{}".format(sym.latex(file)))

In [107]:
(expr + den_constand).subs({p_1:0, p_2:0, p_3:0, p_4:0, q_1:1, q_2:1, q_3:1, q_4:1})